<a href="https://colab.research.google.com/github/vidyashankarappa/DeepLearning/blob/main/DeepLearningLLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets # install datasets
!pip install trl # install transformer reinforcement leanring
!pip install evaluate # install evaluate
!pip install unsloth # install unsloth


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system ==

In [ ]:
import re
from tqdm import tqdm
import torch # Import PyTorch
from datasets import load_dataset # Lets you load fine-tuning datasets
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import TrainingArguments # Defines training hyperparameters
import evaluate # Evaluation
from trl import SFTTrainer # Trainer for supervised fine-tuning (SFT)
from unsloth import FastLanguageModel # Modules for fine-tuning
from unsloth import is_bfloat16_supported # Checks bfloat16 supports

<ipython-input-1-066d22dc7110>:9: UserWarning: WARNING: Unsloth should be imported before trl, transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel # Modules for fine-tuning


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
# Make sure to change runtime to GPU
# Check if GPU is avaiable
device = "cuda" if torch.cuda.is_available() \
else "mps" if torch.mps.is_available() \
else "cpu"
print("Device:", device)


Device: cuda


In [ ]:
dataset = load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT","en",
                       split=["train[:80%]","train[80%:90%]","train[90%:]"])
train_dataset, val_dataset, test_dataset = dataset

train_dataset, val_dataset, test_dataset

README.md:   0%|          | 0.00/1.65k [00:00<?, ?B/s]

medical_o1_sft.json:   0%|          | 0.00/74.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25371 [00:00<?, ? examples/s]

(Dataset({
     features: ['Question', 'Complex_CoT', 'Response'],
     num_rows: 20297
 }),
 Dataset({
     features: ['Question', 'Complex_CoT', 'Response'],
     num_rows: 2537
 }),
 Dataset({
     features: ['Question', 'Complex_CoT', 'Response'],
     num_rows: 2537
 }))

In [ ]:
train_dataset_small = train_dataset.shuffle().select(range(1000))
val_dataset_small = val_dataset.select(range(100))
test_dataset_small = test_dataset.select(range(100))

In [ ]:
import pandas as pd # Import pandas for DataFrame conversion
from datasets import load_dataset # Import load_dataset from datasets
df = pd.DataFrame(train_dataset.take(3))
display(df)

,Question,Complex_CoT,Response
0,A 61-year-old woman with a long history of inv...,"Okay, let's think about this step by step. The...",Cystometry in this case of stress urinary inco...
1,A 45-year-old man with a history of alcohol us...,"Alright, let’s break this down. We have a 45-y...",Considering the clinical presentation of sudde...
2,A 45-year-old man presents with symptoms inclu...,"Okay, so here's a 45-year-old guy who's experi...",Based on the clinical findings presented—wide-...


In [ ]:
train_prompt_style = """Below is an instruction that describes a task,
paired with an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response


### Instruction:
You are a medical expert with advanced knowledge in clinical reasoning,
diagnostics, and treatment planning.
Please answer the following medical question.

### Question:
{}

### Response:
<think>
{}
</think>
<answer>
{}
</answer>"""

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("unsloth/DeepSeek-R1-Distill-Llama-8B")
EOS_TOKEN = tokenizer.eos_token # EOS_TOKEN
print(EOS_TOKEN)


<｜end▁of▁sentence｜>


In [ ]:
def formatting_prompts_func(examples, prompt_style):
    inputs = examples["Question"]
    cots = examples["Complex_CoT"]
    outputs = examples["Response"]
    texts = []
    for input, cot, output in zip(inputs, cots, outputs):
        text = prompt_style.format(input, cot, output) + EOS_TOKEN
        texts.append(text)
    return {
        "text": texts,
    }

In [ ]:
# Apply text format
train_dataset_small = train_dataset_small.map(lambda x:
formatting_prompts_func(x,train_prompt_style),
batched=True)


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
# Apply LoRA to the model
model_lora = FastLanguageModel.get_peft_model(
    model,


SyntaxError: closing parenthesis ')' does not match opening parenthesis '[' on line 6 (<ipython-input-26-ab4b8613d175>, line 47)